In [56]:

vcf_file_address= "22_all.vcf"#"22_allps_200.vcf"
vcf_file = open(vcf_file_address,'r');


first_het_variant_s1=True 
blockid_blocks_s1=[]
allele_blocks_s1=[]
varpos_blocks_s1=[]

first_het_variant_s2=True 
blockid_blocks_s2=[]
allele_blocks_s2=[]
varpos_blocks_s2=[]
var_blockids1_blocks_s2=[]   # for each var in s2, shows the s1 blockid


alleles_s5=[]
varposs_s5=[]

header_lines_list=[]
for line in vcf_file:
    line_strip=line.strip()
    if line_strip.startswith('#'):
        header_lines_list.append(line_strip)
        sample_names=line_strip.split('\t')[9:] # last line of header contains sample name
        #print(sample_names)
    else:

        line_parts=line_strip.split('\t')
        gt_flags, s1, sss, ss, s2, s5 = line_parts[8:]

        varpos=int(line_parts[1]) # variant position
        gt_flags_split=gt_flags.split(":")
        
        if "PS" in gt_flags_split:
            idx_ps= gt_flags.split(":").index("PS")
            idx_gt= gt_flags.split(":").index("GT")
            
            
            # parental
            split_s5 = s5.split(":")
            blockid_s5 = split_s5[idx_ps]
            allele_s5 = split_s5[idx_gt]
            if allele_s5 !="./.":
                alleles_s5.append(allele_s5[0]) 
                varposs_s5.append(varpos)

            
            split_s1=s1.split(":")
            blockid_s1 = split_s1[idx_ps]
            allele_s1=split_s1[idx_gt]
            
            if allele_s1=="0/1":allele_s1="./.";blockid_s1="."
                
            if not blockid_s1 == '.':
                if first_het_variant_s1:           # for the first het variant
                    first_het_variant_s1 = False
                    
                    allele_block_s1 = [int(allele_s1[0])]     # alleles of a block
                    varpos_block_s1 = [varpos]               # var pos of a block
                    blockid_blocks_s1.append(int(blockid_s1))   # add the blockid of the first var  to the list of blockids
                    
                else:                              # for the rest of het variants
                    if int(blockid_s1) in blockid_blocks_s1:
                        allele_block_s1.append(int(allele_s1[0]))
                        varpos_block_s1.append(varpos)
                        
                    else: 
                        # add previous block to the list of all blocks
                        allele_blocks_s1.append(allele_block_s1)
                        varpos_blocks_s1.append(varpos_block_s1)

                        # creat new phase block
                        allele_block_s1 = [int(allele_s1[0])]
                        varpos_block_s1 = [varpos]
                        blockid_blocks_s1.append(int(blockid_s1))

                        
            
            split_s2=s2.split(":")
            blockid_s2 = split_s2[idx_ps]
            allele_s2=split_s2[idx_gt]
            
            if allele_s2=="0/1":allele_s2="./.";blockid_s2="."
                
            if not blockid_s2 == '.':
                if first_het_variant_s2:           # for the first het variant
                    first_het_variant_s2 = False
                    
                    allele_block_s2 = [int(allele_s2[0])]     # alleles of a block
                    varpos_block_s2 = [varpos]               # var pos of a block
                    blockid_blocks_s2.append(int(blockid_s2))   # add the blockid of the first var  to the list of blockids
                    var_blockids1_block_s2=[blockid_s1]
                    
                else:                              # for the rest of het variants
                    if int(blockid_s2) in blockid_blocks_s2:
                        allele_block_s2.append(int(allele_s2[0]))
                        varpos_block_s2.append(varpos)
                        var_blockids1_block_s2.append(blockid_s1)
                        
                    else: 
                        # add previous block to the list of all blocks
                        allele_blocks_s2.append(allele_block_s2)
                        varpos_blocks_s2.append(varpos_block_s2)
                        var_blockids1_blocks_s2.append(var_blockids1_block_s2)

                        # creat new phase block
                        allele_block_s2 = [int(allele_s2[0])]
                        varpos_block_s2 = [varpos]
                        blockid_blocks_s2.append(int(blockid_s2))
                        var_blockids1_block_s2=[blockid_s1]
                       
                        
                        
# add last block to the list of all blocks
allele_blocks_s1.append(allele_block_s1)
varpos_blocks_s1.append(varpos_block_s1)

allele_blocks_s2.append(allele_block_s2)
varpos_blocks_s2.append(varpos_block_s2)
var_blockids1_blocks_s2.append(var_blockids1_block_s2)
                        
    
    
number_phaseblocks_s2= len(allele_blocks_s2)
    
    

    
intersected_blocks_idins1_s2=[] # for each block in s2, the corresponding s1 block id

for var_blockids1_blocks_s2_i in var_blockids1_blocks_s2:
    var_blockids1_blocks_s2_i_set=set(var_blockids1_blocks_s2_i)
    
    if '.' in var_blockids1_blocks_s2_i_set:
        var_blockids1_blocks_s2_i_set.discard('.')
        
    if len(var_blockids1_blocks_s2_i_set)==1:
        intersected_idins1_s2=int(list(var_blockids1_blocks_s2_i_set)[0])
    if len(var_blockids1_blocks_s2_i_set)==0:
        intersected_idins1_s2=0
    if len(var_blockids1_blocks_s2_i_set)>1:
        # find the most frequent blockid in the phaseblock
        intersected_idins1_s2=int(max(var_blockids1_blocks_s2_i_set, key=var_blockids1_blocks_s2_i.count) ) 
        
    intersected_blocks_idins1_s2.append(intersected_idins1_s2)

        

In [57]:
print(blockid_blocks_s1)
for i in range(len(allele_blocks_s1)):
    print(len(allele_blocks_s1[i]))
    
print(blockid_blocks_s2)
for i in range(len(allele_blocks_s2)):
    print(len(allele_blocks_s2[i]))

[16065536, 16406342, 16657810, 16960113, 18724634, 18930729, 20489260, 20705103, 21505644, 21734889, 23284023, 23857083, 24438699, 35094331, 39054450, 50432604]
22
14
12
1604
5
1714
5
526
7
412
606
593
8095
2680
10156
760
[16065536, 16067462, 16068688, 16071420, 16072581, 16464422, 16661781, 16664149, 16960113, 16961980, 16964827, 17004454, 17005828, 17008023, 17009231, 17009877, 17010978, 17014115, 17094670, 17097910, 17099462, 17101987, 17103717, 17105001, 17108394, 17110402, 17118076, 17120645, 17122591, 17123971, 17126163, 17127617, 17132490, 17133464, 17140023, 17141389, 17145314, 17152611, 17156061, 17161286, 17222962, 17276424, 17295703, 17296426, 17299901, 17301217, 17302458, 17305777, 17307742, 17310488, 17312596, 17313288, 17314934, 17316555, 17318150, 17320361, 17323001, 17326181, 17327595, 17328627, 17330409, 17333682, 17338687, 17340906, 17341800, 17343037, 17344500, 17346384, 17348229, 17350075, 17352097, 17353830, 17359912, 17387404, 17389912, 17391088, 17392568, 1739345

3
2
7
5
3
8
14
4
5
4
2
2
3
19
3
3
2
24
4
3
4
2
11
6
2
4
2
6
3
2
2
2
9
3
3
5
2
4
2
2
5
2
14
3
5
3
4
6
4
3
4
2
2
4
3
2
2
2
2
2
5
7
4
5
2
2
3
7
10
4
6
9
14
2
2
2
4
2
2
2
3
2
3
3
2
2
5
2
4
2
2
2
2
6
3
2
4
4
2
2
2
2
2
3
2
3
2
2
2
3
2
2
2
2
2
2
2
2
2
2
2
3
2
2
2
2
2
3
3
2
2
2
2
2
2
4
2
2
7
3
2
3
2
2
2
3
2
3
3
2
3
3
5
2
2
2
5
4
2
6
4
2
2
3
5
2
2
3
2
2
5
2
4
4
2
2
2
2
2
2
2
2
2
4
5
2
2
2
2
2
2
5
4
2
7
3
12
2
2
2
2
4
5
2
3
4
3
2
2
2
2
2
2
2
3
5
4
8
2
3
5
3
11
7
6
10
3
2
2
2
2
2
3
2
3
2
3
4
16
3
3
4
2
3
4
3
2
2
4
11
3
2
6
2
2
2
3
2
2
2
2
2
2
2
2
7
2
3
2
2
3
15
2
3
4
2
5
2
2
2
2
2
2
4
6
2
2
3
2
3
3
2
2
2
6
2
3
3
3
2
3
2
2
2
2
3
2
2
2
6
3
3
11
3
2
3
7
3
3
2
2
3
2
2
2
2
4
2
2
4
3
2
6
2
3
2
6
3
7
3
5
5
2
2
8
3
4
2
2
2
2
5
3
5
3
4
2
2
2
3
4
2
3
2
2
2
3
2
2
2
2
2
3
2
3
2
3
2
2
2
2
4
3
2
2
2
2
2
5
2
2
3
2
2
2
2
3
2
7
3
2
2
4
3
2
2
2
3
2
2
2
2
2
2
2
2
2
2
5
2
2
2
4
2
2
2
3
2
2
2
2
3
2
2
3
2
2
3
2
2
3
4
4
3
2
4
2
5
2
4
2
22
6
3
9
2
2
7
3
3
2
2
2
2
4
2
2
2
2
3
2
2
3
2
4
2
2
3
2
4
3
5
2
2
2
3
3
2
2
6
2
7
2

In [58]:

allele_blocks_s2_new=list(allele_blocks_s2)
for block_i in range(number_phaseblocks_s2): # 
#for block_i, allele_block_s2 in enumerate(allele_blocks_s2):
   
    allele_block_s2=allele_blocks_s2[block_i]
    varpos_block_s2=varpos_blocks_s2[block_i]

    intersected_idins1_s2=intersected_blocks_idins1_s2[block_i]
    if intersected_idins1_s2 !=0:
        block_indx_s1= blockid_blocks_s1.index(intersected_idins1_s2)
    varpos_block_s1=varpos_blocks_s1[block_indx_s1]
    allele_block_s1=allele_blocks_s1[block_indx_s1]
    
    intersect_varpos=set(varpos_block_s1) & set(varpos_block_s2)
    similiar_count=0
    for varpos in intersect_varpos:
        allele_s1= allele_block_s1[varpos_block_s1.index(varpos)]
        allele_s2= allele_block_s2[varpos_block_s2.index(varpos)]
        if allele_s1==allele_s2: similiar_count+=1
    #print(similiar_count/len(intersect_varpos))
    if similiar_count < len(intersect_varpos)/2:
        #print("need", similiar_count)
        allele_block_s2_new= [1-i for i in allele_block_s2]
        allele_blocks_s2_new[block_i]=allele_block_s2_new
        


In [59]:
matrix_address="matrix_"+vcf_file_address

matrix_file=open(matrix_address,'w');
matrix_file.write("\t\t".join(["Pos","ONT","Illm","parental"])+"\n") # "Pos","ONT","CLR","HiFi","Illm","parental"


for block_i_s1 in range(len(blockid_blocks_s1)):
    blockid_block_s1=blockid_blocks_s1[block_i_s1]
    
    allele_block_s1=allele_blocks_s1[block_i_s1]
    varpos_block_s1=varpos_blocks_s1[block_i_s1]
    for var_i_s1 in range(len(varpos_block_s1)):
        varpos=varpos_block_s1[var_i_s1]
        allele_s1=allele_block_s1[var_i_s1]
        
        allele_s2='.'
        blockid_s2='.'
        for block_i_s2 in range(len(blockid_blocks_s2)):
            blockid_block_s2 = blockid_blocks_s2[block_i_s2]
            allele_block_s2_new = allele_blocks_s2_new[block_i_s2]
            varpos_block_s2 = varpos_blocks_s2[block_i_s2]
            
            if varpos in varpos_block_s2:
                allele_s2 = allele_block_s2_new[varpos_block_s2.index(varpos)]
                blockid_s2=blockid_block_s2
                break # when var is found, no need to continue for loop
            
        if varpos in varposs_s5:
            allele_s5 = alleles_s5[varposs_s5.index(varpos)]
        else: allele_s5= '.'

                
        matrix_row_list=[str(varpos),str(allele_s1)+":"+str(blockid_block_s1),
                         str(allele_s2)+":"+str(blockid_s2),str(allele_s5)] 
        matrix_file.write("\t".join(matrix_row_list)+"\n")
    
matrix_file.close()


In [60]:
len(blockid_blocks_s1)

16

In [61]:
varposs_s5[0:10]

[16051453,
 16056353,
 16062995,
 16065078,
 16065536,
 16067208,
 16068295,
 16069374,
 16073196,
 16077252]